In [24]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import re # 숫자추출을 위한 라이브러리

# 엑셀 만들기
wb = openpyxl.Workbook()

# 워크시트 만들기
ws = wb.active
ws.title = '하스스톤 전설등급 유저수'

# 데이터 추가하기
ws.append(['시즌','서버','유저수'])

# 엑셀 저장하기
wb.save('./hs_legend.xlsx')

# 서버, 시즌 변수 정의
servers = ['AP','US','EU'] # 아시아태평양,북미,유럽
seasonId = list(range(108,122)) # 108부터 121까지 시즌

wb = openpyxl.load_workbook('./hs_legend.xlsx')
ws = wb.active

In [26]:
from time import sleep

for season in seasonId:
  for server in servers:
    response = requests.get(f"https://hearthstone.blizzard.com/ko-kr/community/leaderboards?region={server}&leaderboardId=standard&seasonId={season}")
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # metadata 클래스를 갖는 div 태그들 중에서 h2 태그를 찾음
    metadata_divs = soup.select('.metadata')
    print("metadata_divs", metadata_divs)
        
    for metadata_div in metadata_divs:
      h2_tag = metadata_div.select_one('h2')

      if h2_tag:
        # h2 태그 내의 텍스트에서 숫자만 추출
        num_text = h2_tag.text
        print("num_text", num_text)
        num_match = re.search(r'\d+', num_text)

        if num_match:
            num = num_match.group()
            ws.append([season, server, num])
    sleep(1)


wb.save('./hs_legend.xlsx')